In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-27"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
379,2024-06-27,Eua Wnba,20:00,Chicago Sky F,Las Vegas Aces F,5.40,1.18,170.5,1.91,1.91,9.5,2.05,1.02,pz7KMP3l,0.185185,0.847458,0.523560,0.523560,0.032643,0.0,0.0,NaN,NaN,167.518,48.486591,0.289441,NaN,NaN,NaN,0.0,0.0,243.00,0.00,1.0,0.0,0.0,0.0,0.906988,0.000000,0.474476,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
380,2024-06-27,Eua Wnba,20:00,Washington Mystics F,Connecticut Sun F,5.15,1.18,152.5,1.80,1.95,9.5,1.95,1.02,E9BOLqJf,0.194175,0.847458,0.555556,0.512821,0.041632,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,178.45,106.24,0.0,0.0,0.0,0.0,0.886955,0.056569,0.442835,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
381,2024-06-27,Europa Campeonato Das Nações Europeias Menores,12:00,Gibraltar,São Marino,1.90,1.82,144.5,1.87,1.83,-1.5,1.99,1.92,86kR0HU9,0.526316,0.549451,0.534759,0.546448,0.075766,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.030413,0.015289,0.025318,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
382,2024-06-27,México Cibacopa,23:00,Rayos de Hermosillo,Astros,1.98,1.70,181.5,1.84,1.82,1.5,1.85,1.61,z7hHa3ll,0.505051,0.588235,0.543478,0.549451,0.093286,0.0,0.0,NaN,NaN,158.972,16.091927,0.101225,NaN,NaN,NaN,0.0,0.0,163.20,186.12,0.0,0.0,0.0,0.0,0.107603,0.007728,0.098096,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
383,2024-06-27,Porto Rico Bsn,21:00,Capitanes De Arecibo,Mets de Guaynabo,1.76,1.97,179.5,1.83,1.83,-2.5,1.95,1.95,t0LZBSwI,0.568182,0.507614,0.546448,0.546448,0.075796,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.00,132.13,0.0,0.0,0.0,0.0,0.079621,0.000000,0.000000,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,12:00,Europa Campeonato Das Nações Europeias Menores,Gibraltar,São Marino,144.5,1.87,0.9993,Over
1,21:00,Porto Rico Bsn,Capitanes De Arecibo,Mets de Guaynabo,179.5,1.83,0.9820,Over
